In [1]:
import pandas as pd

In [2]:
import numpy as np

In [125]:
names = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item', sep='|', names=names,encoding='latin-1')

In [113]:
col_names = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('u1.base', sep='\t', names=col_names, encoding='latin-1')
ratings_test = pd.read_csv('u1.test', sep='\t', names=col_names, encoding='latin-1')


In [116]:
ratings_base.drop('unix_timestamp',axis=1,inplace=True)

In [117]:
ratings_test.drop('unix_timestamp',axis=1,inplace=True)

In [130]:
movie_data = items.filter(['movie id','movie title'],axis=1)

In [135]:
ratings_base.head()

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [38]:
movie_data.isna().sum()

0    0
1    0
2    0
dtype: int64

In [132]:
ratings_base.isna().sum()

user_id     0
movie_id    0
rating      0
dtype: int64

In [133]:
ratings_test.isna().sum()

user_id     0
movie_id    0
rating      0
dtype: int64

In [134]:
movie_data.rename(columns={'movie title':'title','movie id':'movie_id'},inplace=True)

In [7]:
ratings.rename(columns={0:'user_id',1:'movie_id',2:'rating'},inplace=True)

In [143]:
user_item_pivot = ratings_base.pivot('user_id','movie_id','rating').fillna(0)

In [144]:
user_item_pivot.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
user_similarity = cosine_similarity(user_item_pivot)

In [146]:
def movie_recommender(user_item_pivot, user_similarity, user, k=20, top=10):
    # Get location of the actual movie in the User-Items matrix
    uindex = user_item_pivot.index.get_loc(user)
    # Use it to index the User similarity matrix
    similar_users = user_similarity[uindex]
    # obtain the indices of the top k most similar users
    similar_users = user_item_pivot.index[similar_users.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    movies = user_item_pivot.loc[similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    seen_movies = user_item_pivot.loc[user].gt(0)
    seen_movies = seen_movies.index[seen_movies].tolist()
    top_movies = movies.drop(seen_movies).head(top)
    top_movies = pd.DataFrame(top_movies).reset_index().rename(columns={0:'Mean_Rating'}).head(top)
    # return recommendations - top similar users rated movies
    #return top_movies.index.to_frame().reset_index(drop=True)
    return top_movies.merge(movie_data, how='inner', on='movie_id')

In [95]:
def test_recommender(user_item_pivot, user_similarity, user, k=20, top=10):
    # Get location of the actual movie in the User-Items matrix
    uindex = user_item_pivot.index.get_loc(user)
    # Use it to index the User similarity matrix
    similar_users = user_similarity[uindex]
    # obtain the indices of the top k most similar users
    similar_users = user_item_pivot.index[similar_users.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    movies = user_item_pivot.loc[similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    #seen_movies = user_item_pivot.loc[user].gt(0)
    #seen_movies = seen_movies.index[seen_movies].tolist()
    top_movies = movies.head(top)
    top_movies = pd.DataFrame(top_movies).reset_index().rename(columns={0:'Mean_Rating'}).head(top)
    # return recommendations - top similar users rated movies
    #return top_movies.index.to_frame().reset_index(drop=True)
    return top_movies.merge(movie_data, how='inner', on='movie_id')

In [157]:
movies.drop(seen_movies).isin(test).sum()

0

In [180]:
res = movie_recommender(user_item_pivot,user_similarity,1)

In [174]:
unseen = pd.Series(ratings_test[ratings_test['user_id']==1]['movie_id'])

In [181]:
res['movie_id']

0    174
1     96
2     98
3     12
4    100
5     56
6     69
7     64
8    423
9    183
Name: movie_id, dtype: int64

In [185]:
ratings_test[(ratings_test['movie_id'].isin(res['movie_id']))&(ratings_test['user_id']==1)]

,user_id,movie_id,rating
2,1,12,5
19,1,56,4
23,1,64,5
26,1,69,3
42,1,96,5
44,1,98,4
45,1,100,5
80,1,174,5
84,1,183,5


In [ ]:
# to check the recommendations
test_user = ratings[ratings['user_id']==12]

In [ ]:
test_user.sort_values(by='rating',ascending=False)[:10]

In [16]:
data = ratings[(ratings['user_id']>=1) &(ratings['user_id']<=500)]

In [18]:
pearson_data = data.pivot('movie_id','user_id','rating').fillna(0)

In [35]:
pearson_similarities = pearson_data.corr(method='pearson').to_numpy()

In [30]:
def pearson_recommender(user_item_pivot, user_similarity, user, k=20, top=10):
    # Get location of the actual movie in the User-Items matrix
    uindex = user_item_pivot.index.get_loc(user)
    # Use it to index the User similarity matrix
    similar_users = user_similarity[uindex]
    # obtain the indices of the top k most similar users
    similar_users = user_item_pivot.index[similar_users.argpartition(-k)[-k:]]
    # Obtain the mean ratings of those users for all movies
    movies = user_item_pivot.loc[similar_users].mean(0).sort_values(ascending=False)
    # Discard already seen movies
    seen_movies = user_item_pivot.loc[user].gt(0)
    seen_movies = seen_movies.index[seen_movies].tolist()
    top_movies = movies.drop(seen_movies).head(top)
    # return recommendations - top similar users rated movies
    return top_movies.index.to_frame().reset_index(drop=True)

In [59]:
k = pd.Series([True,False,True])

In [60]:
k.index[k]

Int64Index([0, 2], dtype='int64')

In [38]:
pearson_result = pearson_recommender(data.pivot('user_id','movie_id','rating').fillna(0),pearson_similarities,12)

In [39]:
pearson_result = pearson_result.merge(movie_data,on='movie_id',how='inner')

In [40]:
pearson_result

,movie_id,title,genres
0,2683,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy
1,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2,2716,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Fantasy|Sci-Fi
3,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
4,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,2706,American Pie (1999),Comedy
6,260,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,Action|Adventure|Sci-Fi
7,2028,Saving Private Ryan (1998),Action|Drama|War
8,3793,X-Men (2000),Action|Adventure|Sci-Fi
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
